<a href="https://colab.research.google.com/github/naokityokoyama/HDC/blob/main/HDC_RecordEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch-hd binhd unidecode num2words torchmetrics -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.0/361.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.9/981.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:0

In [3]:
import zipfile
from unidecode import unidecode
import string
from num2words import num2words
import re
import numpy as np
import pandas as pd
from typing import Union, Literal
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torchhd
from torchhd import embeddings

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from binhd.embeddings import ScatterCode
from binhd.datasets import BaseDataset
from binhd.classifiers import BinHD, NeuralHD
from torchmetrics import Accuracy


import warnings
warnings.filterwarnings("ignore")

In [4]:
path_origin = '/content/drive/MyDrive/uff/fake.zip'
path_destino = '/content/'
with zipfile.ZipFile(path_origin, "r") as zip_ref:
    zip_ref.extractall(path_destino)

In [5]:
#build dataset

df_fake = pd.read_csv('/content/fakes.csv')[['text']]
df_fake['target'] = 1
df_true = pd.read_csv('/content/true.csv')[['text']]
df_true['target'] = 0
df = pd.concat([df_fake, df_true]).reset_index(drop=True)

In [6]:
# Definir o tamanho da amostra
sample_size = 10000

# Criar uma amostra balanceada
df = df.groupby("target", group_keys=False).apply(lambda x: resample(x, n_samples=sample_size // df["target"].nunique(), random_state=42))
df = df.reset_index(drop=True)

In [7]:
df.head(3)

,text,target
0,"Entre 2007 e 2016, cerca de 553 mil pessoas mo...",0
1,As propinas descem pela primeira vez em 15 anos?,0
2,"O rendimento ""per capita"" dos portugueses é o ...",0


In [8]:
#clean

def n2w(texto:str)->str:
  padrao = r"\d+"
  numeros = re.findall(padrao, texto)
  for numero in numeros:
    extenso = num2words(numero, lang='pt')
    texto = texto.replace(numero, extenso)
  return texto

In [9]:
for repet in tqdm(range(2)):  #bug para rodar 2x
  df['text'] = df['text'].str.lower()
  df['text'] = df['text'].str.replace(f"[{string.punctuation}]", "", regex=True)
  df['text'] = df['text'].apply(lambda x: ' '.join(x.split()))
  df['text'] = df['text'].str.replace('"', '').str.replace('\\', '')
  df['text'] = df['text'].apply(n2w)
  df['text'] = df['text'].apply(unidecode)

100%|██████████| 2/2 [00:00<00:00,  3.79it/s]


2

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

MAX_INPUT_SIZE = 128
PADDING_IDX = 0

ASCII_A = ord("a")
ASCII_Z = ord("z")
ASCII_SPACE = ord(" ")
NUM_TOKENS = ASCII_Z - ASCII_A + 3  # a through z plus space and padding
print (ASCII_A, '--', ASCII_Z, '--', ASCII_SPACE, '--', NUM_TOKENS)

def char2int(char: str) -> int:
    """Map a character to its integer identifier"""
    ascii_index = ord(char)

    if ascii_index == ASCII_SPACE:
        # Remap the space character to come after "z"
        return ASCII_Z - ASCII_A + 1

    return ascii_index - ASCII_A


def transform(x: str) -> torch.Tensor:
    char_ids = x[:MAX_INPUT_SIZE]
    char_ids = [char2int(char) + 1 for char in char_ids.lower()]

    if len(char_ids) < MAX_INPUT_SIZE:
        char_ids += [PADDING_IDX] * (MAX_INPUT_SIZE - len(char_ids))

    return torch.tensor(char_ids, dtype=torch.long)

Using cpu device
97 -- 122 -- 32 -- 28


In [11]:
#create X and y
lst = []
print ('size dataset',df.shape[0] )
for i in range(df.shape[0]):
  lst.append (np.array(transform(df['text'][i])))

X = np.array(lst)
y = list(df['target'])

size dataset 10000


Record Encoder

In [12]:
# Use the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

dimension = 1000
num_levels = 100
num_feature = X.shape[1]
num_classe = len(set(np.array(y)))

min_val = X[X != 0].min()
max_val = X.max()
print(min_val, max_val)

class RecordEncoder(nn.Module):
    def __init__(self, out_features, size, levels, low, high):
        super(RecordEncoder, self).__init__()
        self.position = embeddings.Random(size, out_features, vsa="BSC", dtype=torch.uint8)
        self.value = ScatterCode(levels, out_features, low = low, high = high)

    def forward(self, x):
        sample_hv = torchhd.bind(self.position.weight, self.value(x))
        sample_hv = torchhd.multiset(sample_hv)
        return sample_hv

Using cpu device
1 27


In [13]:
record_encode = RecordEncoder(dimension, num_feature, num_levels, min_val, max_val)
record_encode = record_encode.to(device)

In [21]:
with torch.no_grad():
    samples = torch.tensor(X).to(device)
    labels = torch.tensor(y).squeeze().to(device)

batch_size = 1000
num_samples = samples.shape[0]

for i in tqdm(range(0, num_samples, batch_size)):
    batch = samples[i:i+batch_size]
    batch_labels = labels[i:i+batch_size]

    X_hv = record_encode(batch)

    X_train, X_test, y_train, y_test = train_test_split(X_hv, batch_labels, test_size=0.3, random_state = 0)

    model = BinHD(dimension, num_classe)


    with torch.no_grad():
      model.fit(X_train,y_train)



100%|██████████| 10/10 [00:13<00:00,  1.38s/it]


In [22]:
predictions = model.predict(X_test)
acc = accuracy_score(predictions, y_test)
print("BinHD - Adapt: Accuracy = ", acc)

BinHD - Adapt: Accuracy =  0.6366666666666667
